In [1]:
import sys,re,random
from bs4 import BeautifulSoup as bs
import pandas as pd
from tqdm import tqdm
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
import nest_asyncio
import aiohttp
import asyncio

In [2]:
nest_asyncio.apply()

# Store URL

In [3]:
urls=[]
main_url="http://staff.family.co.jp/dsaiyo/cofm/search?pc=&cc=cofm&pu=&ac=&cb=all&st=&pcs%5B%5D=01"

# Chrome
options = Options()
driver = webdriver.Chrome(options = options)
driver.get(main_url)
while True:
    try:
        driver.find_element_by_xpath('//*[@id="job-list-readmore"]/form/input[1]').click()
        time.sleep(2)
    except: 
        soup = bs(driver.page_source, "lxml")
        urls_=soup.select('p[class="btn-detail"] > a')
        for i in range(len(urls_)):
            urls.append("http://staff.family.co.jp"+re.findall(r"=\"(.+)\"",str(urls_[i]))[0])
        
        driver.quit()
        break
        
urls=set(urls)

27.53646159172058


# Store name, address, hourly wage

In [4]:
async def fetch(session, url):
    async with session.get(url) as html:
        return await html.text()

In [5]:
async def parser(html):
    soup=bs(html,'lxml')
    #time.sleep(random.uniform(0.5,1.5))
    shop=soup.select('h1')
    if shop:
        sl=soup.select('p[class="salary"]')[0]
        name.append(re.findall(r">(.+)<",str(shop))[0])
        ads.append(re.findall(r">(.+)<",str(soup.select('tr>td')[1]))[0])
        salary.append(re.findall(r"\d*\,*\d+",str(sl))[0])


In [6]:
async def download(url):
    async with aiohttp.ClientSession() as session:
        html = await fetch(session, url)
        await parser(html)

In [ ]:
name=[]
salary=[]
ads=[]

loop = asyncio.get_event_loop()
tasks = [asyncio.ensure_future(download(url)) for url in urls]
tasks = asyncio.gather(*tasks)
loop.run_until_complete(tasks)

In [9]:
df=pd.read_pickle(r"ce.pkl")

In [10]:
shop=["ファミリーマート" for i in range(len(name))]
shop1=pd.DataFrame(shop)
name1=pd.DataFrame(name)
ads1=pd.DataFrame(ads)
salary1=pd.DataFrame(salary)

In [9]:
len(urls)

199

In [14]:
cc=pd.concat([shop1,name1,ads1,salary1],axis=1)
cc.columns=["shop","name","address","salary"]
cc=pd.concat([df,cc],axis=0)
cc.to_pickle(r"ce.pkl")

# Store latitude and longitude, business hours

In [ ]:
lat=[]
lng=[]
tm=[]
main_url="https://www.google.com/maps/"
options = Options()
driver = webdriver.Chrome(options = options)
driver.get(main_url)
for i in tqdm(range(len(name))):
    #google map 検索-入力
    driver.find_element_by_id('searchboxinput').send_keys(name[i])
    driver.find_element_by_id('searchbox-searchbutton').click()
    time.sleep(3)
    lat_=re.findall(r"3d(\d+\.*\d+)!4d",driver.current_url)[0]
    lng_=re.findall(r".+4d(.+)$",driver.current_url)[0]
    lat.append(lat_)
    lng.append(lng_)
    
    tm.append(driver.find_element_by_xpath('//*[@id="pane"]/div/div[1]/div/div/div[12]/div[1]/span[2]/span[2]').text)
    
    driver.find_element_by_id('searchboxinput').clear()
driver.quit()

# Data storage

In [ ]:
name1=pd.DataFrame(name)
ads1=pd.DataFrame(ads)
tm1=pd.DataFrame(tm)
salary1==pd.DataFrame(salary)
lat1=pd.DataFrame(lat)
lng1=pd.DataFrame(lng)

In [ ]:
cc=pd.concat([name1,ads1,tm1,salary1,lat1,lng1],axis=1)
cc.columns=["name","address","business_hours","salary","lat","lng"]
cc.to_csv(r"familymart.csv",encoding="utf_8_sig",index=False)